In [29]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-r-example

cd model

#set -e # stop if anything fails

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon   16.9kB
Step 1/9 : FROM ubuntu:16.04
 ---> 2a697363a870
Step 2/9 : MAINTAINER Amazon SageMaker Examples <amazon-sagemaker-examples@amazon.com>
 ---> Using cache
 ---> 7dc69444bafa
Step 3/9 : RUN apt-get -y update && apt-get install -y --no-install-recommends     wget     r-base     r-base-dev     libcurl4-openssl-dev     libxml2-dev     libssl-dev     libopenblas-dev     liblapack-dev     libopencv-dev     libxt-dev     libpython3-dev     python3-venv     python3     ca-certificates
 ---> Using cache
 ---> 47313c9b7875
Step 4/9 : RUN R -e "install.packages(c('mda', 'plumber','twitteR','ROAuth','tidyverse','purrrlyr','text2vec','caret','glmnet','ggrepel','dummies','reticulate'), repos='https://cloud.r-project.org')"
 ---> Using cache
 ---> 0c307d17310e
Step 5/9 : RUN Rscript -e "install.packages(c('mda', 'plumber'),repos='https://cloud.r-project.org')"
 ---> Using cache
 ---> e42a0ddb8b3b
Step 6/9 : RUN Rscript -e "install.packages

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [30]:
import boto3

import time
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')


In [31]:
r_job = 'sagemaker-r-example-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

print("Training job", r_job)

bucket = "rpoc-master-pipeline-input-data"
prefix = "input/data"

r_training_params = {
    "RoleArn": role,
    "TrainingJobName": r_job,
    "AlgorithmSpecification": {
        "TrainingImage": '{}.dkr.ecr.{}.amazonaws.com/sagemaker-r-example:latest'.format(account, region),
        "TrainingInputMode": "File"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.m4.xlarge",
        "VolumeSizeInGB": 10
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/training".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        },
        {
            "ChannelName": "test",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/testing".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }
    ],
    "OutputDataConfig": {
        "S3OutputPath": "s3://rpoc-master-pipeline-model-output"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60
    }
}

Training job sagemaker-r-example-2019-06-10-13-01-01


In [32]:
%%time

sm = boto3.client('sagemaker')
sm.create_training_job(**r_training_params)

CPU times: user 17.7 ms, sys: 3.84 ms, total: 21.5 ms
Wall time: 271 ms


In [33]:
r_hosting_container = {
    'Image': '{}.dkr.ecr.{}.amazonaws.com/sagemaker-r-example:latest'.format(account, region),
    'ModelDataUrl': sm.describe_training_job(TrainingJobName=r_job)['ModelArtifacts']['S3ModelArtifacts']
}

create_model_response = sm.create_model(
    ModelName=r_job,
    ExecutionRoleArn=role,
    PrimaryContainer=r_hosting_container)

In [35]:
r_endpoint_config = 'sagemaker-r-example-config-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(r_endpoint_config)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=r_endpoint_config,
    ProductionVariants=[{
        'InstanceType': 'ml.m4.xlarge',
        'InitialInstanceCount': 1,
        'ModelName': r_job,
        'VariantName': 'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

sagemaker-r-example-config-2019-06-10-13-08-35
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:001178237879:endpoint-config/sagemaker-r-example-config-2019-06-10-13-08-35


In [ ]:
%%time

r_endpoint = 'sagemaker-r-example-endpoint-' + time.strftime("%Y%m%d%H%M", time.gmtime())
print(r_endpoint)
create_endpoint_response = sm.create_endpoint(
    EndpointName=r_endpoint,
    EndpointConfigName=r_endpoint_config)
print(create_endpoint_response['EndpointArn'])

resp = sm.describe_endpoint(EndpointName=r_endpoint)
status = resp['EndpointStatus']
print("Status: " + status)

try:
    sm.get_waiter('endpoint_in_service').wait(EndpointName=r_endpoint)
finally:
    resp = sm.describe_endpoint(EndpointName=r_endpoint)
    status = resp['EndpointStatus']
    print("Arn: " + resp['EndpointArn'])
    print("Status: " + status)

    if status != 'InService':
        raise Exception('Endpoint creation did not succeed')

sagemaker-r-example-endpoint-201906101309
arn:aws:sagemaker:us-east-1:001178237879:endpoint/sagemaker-r-example-endpoint-201906101309
Status: Creating
